In [ ]:
import numpy as np
from pr3_utils import *
from stereo import *
from tqdm import tqdm

In [ ]:
dataset = "10"
time_stamp,features,linear_velocity,angular_velocity,K,b,imu_T_cam = load_data(f"../data/{dataset}.npz")

num_time_stamp = time_stamp.shape[0]
num_features = features.shape[1]

tau = time_stamp[1:] - time_stamp[:-1]
velocity = np.concatenate([linear_velocity, angular_velocity], axis=1)

T_mean = np.load(f"../data/predict_T_mean_{dataset}.npy")
T_covar = np.load(f"../data/predict_T_covar_{dataset}.npy")

M_init = np.load(f"../data/M_init_{dataset}.npy")

In [ ]:
cam_T_imu = inversePose(imu_T_cam)
fsu = K[0,0]
fsv = K[1,1]
cu  = K[0,2]
cv  = K[1,2]
Ks = np.array([
    [fsu,0,cu,0],
    [0,fsv,cv,0],
    [fsu,0,cu,-fsu*b],
    [0,fsv,cv,0],
])
P = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
])

In [ ]:
# features_mu = np.zeros([num_features*3])
M_mean = np.copy(M_init)
M_covar = np.eye(3*num_features, 3*num_features, dtype=np.float16).reshape(num_features, 3, num_features, 3)

M_covar_norm_t = []
M_covar_norm = []

V = 1
for t in tqdm(range(1, time_stamp.shape[0])):
    present_mask = get_seeing_mask(features[t,:,:], d_min=3)
    Nt = np.sum(present_mask)
    
    # compose dynamic state, covar
    present_M_mean = M_mean[present_mask, :]
    present_M_mean_homo = np.hstack([present_M_mean, np.ones([present_M_mean.shape[0],1])])
    ixgrid = np.ix_(present_mask, np.arange(3), present_mask, np.arange(3))
    present_M_covar = M_covar[ixgrid].reshape([Nt*3, Nt*3])

    # Iterate over these indices
    H = np.zeros([Nt, 4, Nt, 3])
    tt = cam_T_imu @ inversePose(T_mean[t])
    for j in range(Nt):
        H[j,:,j,:] = Ks @ \
         projectionJacobian(tt @ present_M_mean_homo[j,:]) @ \
         tt@P.T
    H = H.reshape([Nt*4, Nt*3])
    K = present_M_covar @ H.T @ np.linalg.inv(H @ present_M_covar @ H.T + V*np.eye(Nt*4))
    
    innovation = features[t,present_mask,:] - (projection(present_M_mean_homo @ tt.T)@Ks.T)
    M_mean[present_mask, :] = (
        present_M_mean.reshape(-1) + K @ innovation.reshape(-1)
    ).reshape(-1, 3)
    M_covar[ixgrid] = (
        (np.eye(Nt*3) - K@H)@present_M_covar
    ).reshape(Nt, 3, Nt, 3)

    # log covariance
    if t%50 == 0:
        M_covar_norm_t.append(time_stamp[t])
        M_covar_norm.append(np.linalg.norm(M_covar))

In [ ]:
fig,_ = visualize_trajectory(T_mean, features=M_mean, show_ori=False, path_name="EKF_localization")
fig.savefig(f"../img/EKF_mapping_{dataset}", dpi=300)
plt.plot()

In [ ]:
plt.plot(M_covar_norm_t, M_covar_norm)
plt.scatter(M_covar_norm_t, M_covar_norm)
plt.xlabel("time stamp")
plt.ylabel("|| M_covar ||")
plt.savefig(f"../img/M_covar_norm_{dataset}", dpi=300)
plt.show()

In [ ]:
from matplotlib.colors import Normalize

sample_step = 400
size = int(np.ceil(num_features/sample_step)*3)
# vmin, vmax = np.min(M_covar), np.max(M_covar)
vmin, vmax = -1, 1
plt.imshow(
    M_covar[::sample_step, :, ::sample_step, :].reshape(size, size), 
    cmap='viridis', origin='lower', norm=Normalize(vmin=vmin, vmax=vmax))
plt.colorbar()
plt.savefig(f"../img/M_covar_sample_{dataset}", dpi=300)
plt.show()

In [ ]:
sample_step = 10
plt.imshow(
    M_covar[:sample_step, :, :sample_step, :].reshape(3*sample_step, 3*sample_step), 
    cmap='viridis', origin='lower', norm=Normalize(vmin=vmin, vmax=vmax))
plt.colorbar()
plt.savefig(f"../img/M_covar_{dataset}(first{sample_step}).png", dpi=300)
plt.show()

In [ ]:
print(M_covar.nbytes/1024/1024/1024, "GB")
np.sum(M_covar<0)

### reshape trick

In [ ]:
# f = np.arange(6).reshape(2,3)
# print(f, f.reshape(-1,1))
present_m = np.array([True, False, True])

cov = np.eye(9).reshape(3,3,3,3)
cov_full = cov.reshape(9,9)
print(cov_full)
cov[present_m,:,present_m,:]